In [58]:
# Some of the functions below require an older version of torchtext than the default one Kaggle gives you.
# IMPORTANT: Make sure that Internet is turned on!!! (Notebook options in the bar on the right)
# IMPORTANT: If you're not already using Kaggle, we STRONGLY recommend you switch to Kaggle for hw1b in particular,
# because copying our notebook will pin you to a Python version that lets you install the right version of torchtext.
# On Colab you will have to downgrade your Python to e.g., 3.7 to do the below pip install, which is a pain to do.
#!pip install torchtext==0.10.0
#exit()


In [1]:
# This block handles some basic setup and data loading.  
# You shouldn't need to edit this, but if you want to 
# import other standard python packages, that is fine.

# imports
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

from torchtext.legacy import data
from torchtext.legacy import datasets

# download and load the data
text_field = data.Field()
datasets = datasets.WikiText2.splits(root='.', text_field=text_field)

train_dataset, validation_dataset, test_dataset = datasets

text_field.build_vocab(train_dataset, validation_dataset, test_dataset)
vocab = text_field.vocab
vocab_size = len(vocab)

train_text = train_dataset.examples[0].text # a list of tokens (strings)
validation_text = validation_dataset.examples[0].text

print(validation_text[:30])

['<eos>', '=', 'Homarus', 'gammarus', '=', '<eos>', '<eos>', 'Homarus', 'gammarus', ',', 'known', 'as', 'the', 'European', 'lobster', 'or', 'common', 'lobster', ',', 'is', 'a', 'species', 'of', '<unk>', 'lobster', 'from', 'the', 'eastern', 'Atlantic', 'Ocean']


In [70]:
print(text_field.vocab.freqs['.'])
print(len(train_text))
print(len(validation_text))

90077
2088628
217646


In [2]:
from collections import defaultdict, Counter
import numpy as np
import math
import tqdm
import random
import pdb

import torch
from torch import nn
import torch.nn.functional as F
import torchtext

We've implemented a unigram model here as a demonstration.

In [3]:
class UnigramModel:
    def __init__(self, train_text):
        self.counts = Counter(train_text)
        self.total_count = len(train_text)

    def probability(self, word):
        return self.counts[word] / self.total_count

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""
        return [self.probability(word) for word in vocab.itos]

    def perplexity(self, full_text):
        """Return the perplexity of the model on a text as a float.
        
        full_text -- a list of string tokens
        """
        log_probabilities = []
        for word in full_text:
            # Note that the base of the log doesn't matter 
            # as long as the log and exp use the same base.
            log_probabilities.append(math.log(self.probability(word), 2))
        return 2 ** -np.mean(log_probabilities)

unigram_demonstration_model = UnigramModel(train_text)
print('unigram validation perplexity:', 
      unigram_demonstration_model.perplexity(validation_text))

def check_validity(model):
    """Performs several sanity checks on your model:
    1) That next_word_probabilities returns a valid distribution
    2) That perplexity matches a perplexity calculated from next_word_probabilities

    Although it is possible to calculate perplexity from next_word_probabilities, 
    it is still good to have a separate more efficient method that only computes 
    the probabilities of observed words.
    """

    log_probabilities = []
    for i in range(10):
        prefix = validation_text[:i]
        probs = model.next_word_probabilities(prefix)
        assert min(probs) >= 0, "Negative value in next_word_probabilities"
        assert max(probs) <= 1 + 1e-8, "Value larger than 1 in next_word_probabilities"
        assert abs(sum(probs)-1) < 1e-4, "next_word_probabilities do not sum to 1"

        word_id = vocab.stoi[validation_text[i]]
        selected_prob = probs[word_id]
        log_probabilities.append(math.log(selected_prob))

    perplexity = math.exp(-np.mean(log_probabilities))
    your_perplexity = model.perplexity(validation_text[:10])
    assert abs(perplexity-your_perplexity) < 0.1, "your perplexity does not " + \
    "match the one we calculated from `next_word_probabilities`,\n" + \
    "at least one of `perplexity` or `next_word_probabilities` is incorrect.\n" + \
    f"we calcuated {perplexity} from `next_word_probabilities`,\n" + \
    f"but your perplexity function returned {your_perplexity} (on a small sample)."


check_validity(unigram_demonstration_model)

unigram validation perplexity: 965.0860734119312


To generate from a language model, we can sample one word at a time conditioning on the words we have generated so far.

In [69]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)

print(generate_text(unigram_demonstration_model))

<eos> <eos> <eos> east of unusual April chapel pages <eos> novel <eos> = The episodes , Let territory the ' Commission <eos>


In fact there are many strategies to get better-sounding samples, such as only sampling from the top-k words or sharpening the distribution with a temperature.  You can read more about sampling from a language model in this recent paper: https://arxiv.org/pdf/1904.09751.pdf.

## Try to implement top-k and top-p sampling (Extension task)

In [64]:
import numpy as np

def generate_text(model, n=20, prefix=('<eos>', '<eos>'), do_sampling=True, top_p=None, top_k=None):
    prefix = list(prefix)

    if not do_sampling:
        for _ in range(n):
            probs = model.next_word_probabilities(prefix)
            word = random.choices(vocab.itos, probs)[0]
            prefix.append(word)
        return ' '.join(prefix)

    
    if do_sampling and top_k != None:
        for _ in range(n):
            probs = model.next_word_probabilities(prefix)
            top_k_probs, top_k_indices = zip(*sorted([(prob, i) for i, prob in enumerate(probs)], reverse=True)[:top_k])
            top_k_probs = np.array(top_k_probs) / sum(top_k_probs)
            word_idx = random.choices(top_k_indices, weights=top_k_probs)[0]
            word = vocab.itos[word_idx]
            prefix.append(word)
        return ' '.join(prefix)

    
    if do_sampling and top_p != None:
        for _ in range(n):
            probs = model.next_word_probabilities(prefix)
            sorted_probs = sorted(enumerate(probs), key=lambda x: x[1], reverse=True)
            cumulative_probs = 0.0
            for i, (index, prob) in enumerate(sorted_probs):
                cumulative_probs += prob
                if cumulative_probs >= top_p or i == len(sorted_probs) - 1:
                    break
            normalized_probs = [prob / cumulative_probs for _, prob in sorted_probs[:i+1]]
            word = random.choices([vocab.itos[index] for index, _ in sorted_probs[:i+1]], weights=normalized_probs)[0]
            prefix.append(word)
        return ' '.join(prefix)
        

print(generate_text(unigram_demonstration_model, do_sampling=True, top_p=0.8))
print(generate_text(unigram_demonstration_model, do_sampling=True, top_k=30))







# Question-1: can we use top-p and top-k together? How to implement it. 

def generate_text_topall(model, n=20, prefix=('<eos>', '<eos>'), top_p=0.9, top_k=30):
    prefix = list(prefix)

    
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        
        # Apply top-p sampling
        sorted_probs = sorted(enumerate(probs), key=lambda x: x[1], reverse=True)
        cumulative_probs = 0.0
        top_p_indices = []
        for i, (index, prob) in enumerate(sorted_probs):
            cumulative_probs += prob
            top_p_indices.append(index)
            if cumulative_probs >= top_p or i == len(sorted_probs) - 1:
                break
        
        # Apply top-k sampling on the top-p subset
        top_p_probs = [probs[idx] for idx in top_p_indices]
        top_k_probs, top_k_indices = zip(*sorted([(prob, top_p_indices[i]) for i, prob in enumerate(top_p_probs)], reverse=True)[:top_k])
        top_k_probs = np.array(top_k_probs) / sum(top_k_probs)
        word_idx = random.choices(top_k_indices, weights=top_k_probs)[0]
        word = vocab.itos[word_idx]

        
        prefix.append(word)
        
    return ' '.join(prefix)





print(generate_text_topall(unigram_demonstration_model, top_p=0.8, top_k=30))


# * Feel free to search on the Internet.
# Question-2: Temperature is also an important parameter in current sampling designs. Can we implement it here?

def generate_text_temperature(model, n=20, prefix=('<eos>', '<eos>'), temperature=1.0):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        probs_smoothed = [float(x)+1e-10 for x in probs] # 平滑处理，避免出现0
        scaled_probs = np.log(probs_smoothed) / temperature
        scaled_probs -= np.max(scaled_probs)
        scaled_probs = np.exp(scaled_probs)
        scaled_probs /= np.sum(scaled_probs)
        word = np.random.choice(vocab.itos, p=scaled_probs)
        prefix.append(word)
    return ' '.join(prefix)
        

print(generate_text_temperature(unigram_demonstration_model, temperature=1.5))

<eos> <eos> that its ( and at , Among , the poem Oxford 1944 were of to to known , has 14
<eos> <eos> for in The . <unk> a for , the of <unk> = ) of . and 's <unk> = "
<eos> <eos> , , ) " and as to with , the to as , 's the a @-@ ) the ,
<eos> <eos> reverse km campuses St. nearly this Fairies governments fungi 220 gave some starts festivities HBO medals Grade Mycena you Bree


You will need to submit some outputs from the models you implement for us to grade.  The following function will be used to generate the required output files.

In [3]:
# !curl -O https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes.txt
# !curl -O https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab.txt
# !curl -O https://cal-cs288.github.io/sp21/project_files/proj_1/eval_prefixes_short.txt
# !curl -O https://cal-cs288.github.io/sp21/project_files/proj_1/eval_output_vocab_short.txt

import tqdm

def save_truncated_distribution(model, filename, short=True):
    """Generate a file of truncated distributions.
    
    Probability distributions over the full vocabulary are large,
    so we will truncate the distribution to a smaller vocabulary.

    Please do not edit this function
    """
    vocab_name = 'eval_output_vocab'
    prefixes_name = 'eval_prefixes'

    if short: 
      vocab_name += '_short'
      prefixes_name += '_short'

    with open('{}.txt'.format(vocab_name), 'r',encoding='utf-8') as eval_vocab_file:
        eval_vocab = [w.strip() for w in eval_vocab_file]
    eval_vocab_ids = [vocab.stoi[s] for s in eval_vocab]

    all_selected_probabilities = []
    with open('{}.txt'.format(prefixes_name), 'r',encoding='utf-8') as eval_prefixes_file:
        lines = eval_prefixes_file.readlines()
        for line in tqdm.notebook.tqdm(lines, leave=False):
            prefix = line.strip().split(' ')
            probs = model.next_word_probabilities(prefix)
            selected_probs = np.array([probs[i] for i in eval_vocab_ids], dtype=np.float32)
            all_selected_probabilities.append(selected_probs)

    all_selected_probabilities = np.stack(all_selected_probabilities)
    np.save(filename, all_selected_probabilities)
    print('saved', filename)

In [66]:
save_truncated_distribution(unigram_demonstration_model, 
                            'unigram_demonstration_predictions.npy')

  0%|          | 0/1000 [00:00<?, ?it/s]

saved unigram_demonstration_predictions.npy


### N-gram Model

Now it's time to implement an n-gram language model.

Because not every n-gram will have been observed in training, use add-alpha smoothing to make sure no output word has probability 0.

$$P(w_2|w_1)=\frac{C(w_1,w_2)+\alpha}{C(w_1)+N\alpha}$$

where $N$ is the vocab size and $C$ is the count for the given bigram.  An alpha value around `3e-3`  should work.  Later, we'll replace this smoothing with model backoff.

One edge case you will need to handle is at the beginning of the text where you don't have `n-1` prior words.  You can handle this however you like as long as you produce a valid probability distribution, but just using a uniform distribution over the vocabulary is reasonable for the purposes of this project.

A properly implemented bi-gram model should get a perplexity below 510 on the validation set.

**Note**: Do not change the signature of the `next_word_probabilities` and `perplexity` functions.  We will use these as a common interface for all of the different model types.  Make sure these two functions call `n_gram_probability`, because later we are going to override `n_gram_probability` in a subclass. 
Also, we suggest pre-computing and caching the counts $C$ when you initialize `NGramModel` for efficiency. 

In [67]:
import nltk

def compute_freq(tokens, n_value=2):
    assert n_value >= 0
    if n_value == 0:
        return {(): len(tokens)}

    ngrams = nltk.ngrams(tokens, n_value)
    ngram_fdist = nltk.FreqDist(ngrams)
    return ngram_fdist


class NGramModel(UnigramModel):
    def __init__(self, train_text, n=2, alpha=3e-3):
        super().__init__(train_text)

        # get counts and perform any other setup
        self.n = n
        self.smoothing = alpha

        # YOUR CODE HERE

        self.ngram_counts = compute_freq(train_text, n)
        self.base_counts = compute_freq(train_text, n-1)

    def n_gram_probability(self, n_gram):
        """Return the probability of the last word in an n-gram.
        
        n_gram -- a list of string tokens
        returns the conditional probability of the last token given the rest.
        """
        assert len(n_gram) == self.n

        # YOUR CODE HERE

        return (self.ngram_counts[tuple(n_gram)] + self.smoothing) / (
                self.base_counts[tuple(n_gram[:-1])] + len(vocab) * self.smoothing)

    def next_word_probabilities(self, text_prefix):
        """Return a list of probabilities for each word in the vocabulary."""

        # YOUR CODE HERE
        # use your function n_gram_probability
        # vocab.itos contains a list of words to return probabilities for

        if len(text_prefix) < (self.n-1):
            return [self.probability(word) for word in vocab.itos]

        prefix = text_prefix[(1-self.n):] if self.n > 1 else []
        return [self.n_gram_probability(prefix+[word]) for word in vocab.itos]

    def perplexity(self, full_text):
        """ full_text is a list of string tokens
        return perplexity as a float """

        # YOUR CODE HERE
        # use your function n_gram_probability
        # This method should differ a bit from the example unigram model because 
        # the first n-1 words of full_text must be handled as a special case.

        log_probabilities = []
        for i in range(self.n-1):
            word = full_text[i]
            log_probabilities.append(math.log(self.probability(word), 2))
        for i in range(self.n-1, len(full_text)):
            n_gram = full_text[(i+1-self.n):(i+1)]
            log_probabilities.append(math.log(self.n_gram_probability(n_gram), 2))
        return 2 ** -np.mean(log_probabilities)


unigram_model = NGramModel(train_text, 1)
check_validity(unigram_model)
print('unigram validation perplexity:', unigram_model.perplexity(validation_text)) # this should be the almost the same as our unigram model perplexity above

bigram_model = NGramModel(train_text, n=2)
check_validity(bigram_model)
print('bigram validation perplexity:', bigram_model.perplexity(validation_text))

trigram_model = NGramModel(train_text, n=3)
check_validity(trigram_model)
print('trigram validation perplexity:', trigram_model.perplexity(validation_text)) # this won't do very well...

#save_truncated_distribution(bigram_model, 'bigram_predictions.npy') # this might take a few minutes

unigram validation perplexity: 965.0913686618096
bigram validation perplexity: 504.4148539982192
trigram validation perplexity: 2965.494861349281


Please download `bigram_predictions.npy` once you finish this section so that you can submit it.

In the block below, please report your bigram validation perplexity.  (We will use this to help us calibrate our scoring on the test set.)

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Bigram validation perplexity: ***504.4148539982192***

We can also generate samples from the model to get an idea of how it is doing.

In [71]:
def generate_text(model, n=20, prefix=('<eos>', '<eos>')):
    prefix = list(prefix)
    for _ in range(n):
        probs = model.next_word_probabilities(prefix)
        word = random.choices(vocab.itos, probs)[0]
        prefix.append(word)
    return ' '.join(prefix)
    
print(generate_text(bigram_model))

<eos> <eos> Gaines have been exaggerated Barbarian definite body but one game of a tick hypocrisy 1740 1783 alone , although he


We now free up some RAM, **it is important to run the cell below, otherwise you may quite possibly run out of RAM in the runtime.**

In [72]:
# Free up some RAM. 
del bigram_model
del trigram_model

This basic model works okay for bigrams, but a better strategy (especially for higher-order models) is to use backoff.  Implement backoff with absolute discounting.
$$P\left(w_i|w_{i-n+1:i-1}\right)=\frac{max\left\{C(w_{i-n+1:i})-\delta,0\right\}}{\sum_{w' \in V} C(w_{i-n+1:i-1}, w')} + \alpha(w_{i-n+1:i-1}) P(w_i|w_{i-n+2:i-1})$$

$$\alpha\left(w_{i-n+1:i-1}\right)=\frac{\delta N_{1+}(w_{i-n+1:i-1})}{{\sum_{w' \in V} C(w_{i-n+1:i-1}, w')}}$$
where $V$ is the vocab and $N_{1+}$ is the number of words that appear after the previous $n-1$ words (the number of times the max will select something other than 0 in the first equation). $w_{i-n+1:i}$ denotes the $n$-gram starting at $w_{i-n+1}$ and ending at $w_i$, and $(w_{i-n+1:i-1}, w')$ denotes the n-gram containing the previous $n-1$ words followed by $w'$. If $\sum_{w' \in V} C(w_{i-n+1:i-1}, w')=0$, use the lower order model probability directly (the above equations would have a division by 0).

We found a discount $\delta$ of 0.9 to work well based on validation performance.  A trigram model with this discount value should get a validation perplexity below 275.

In [73]:
class DiscountBackoffModel(NGramModel):
    def __init__(self, train_text, lower_order_model, n=2, delta=0.9):
        # get counts and perform any other setup
        self.n = n
        self.lower_order_model = lower_order_model
        self.discount = delta

        # YOUR CODE HERE

        self.counts = Counter(train_text)
        self.total_count = len(train_text)

        self.ngram_counts = nltk.FreqDist()
        for i in range(len(train_text)-n):
            prefix = tuple(train_text[i:(i+n-1)])
            if prefix not in self.ngram_counts:
                self.ngram_counts[prefix] = nltk.FreqDist()
            last = train_text[i+n-1]
            if last not in self.ngram_counts[prefix]:
                self.ngram_counts[prefix][last] = 1
            else:
                self.ngram_counts[prefix][last] += 1
        self.base_counts = compute_freq(train_text, n-1)

    def n_gram_probability(self, n_gram):
        assert len(n_gram) == self.n

        # YOUR CODE HERE
        # back off to the lower_order model with n'=n-1 using its n_gram_probability function

        lower_order_prob = self.lower_order_model.n_gram_probability(n_gram[1:])

        ngram = tuple(n_gram)
        last = ngram[-1]
        prefix = ngram[:-1]

        prefix_counts = self.base_counts[prefix]
        if prefix_counts == 0:
            return lower_order_prob

        alpha = len(self.ngram_counts[prefix]) / prefix_counts * self.discount

        return (max([self.ngram_counts[prefix][ngram[-1]] - self.discount, 0]) / prefix_counts + alpha * lower_order_prob)



bigram_backoff_model = DiscountBackoffModel(train_text, unigram_model, 2)
trigram_backoff_model = DiscountBackoffModel(train_text, bigram_backoff_model, 3)
check_validity(trigram_backoff_model)
print('trigram backoff validation perplexity:', trigram_backoff_model.perplexity(validation_text))

trigram backoff validation perplexity: 271.10633787343585


Free up RAM. 

In [74]:
# Release models we don't need any more. 
del unigram_model
del bigram_backoff_model
del trigram_backoff_model

Fill in your trigram backoff perplexity here.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Trigram backoff validation perplexity: ***271.10633787343585***



Free up RAM. 

### Neural N-gram Model

In this section, you will implement a neural version of an n-gram model.  The model will use a simple feedforward neural network that takes the previous `n-1` words and outputs a distribution over the next word.

You will use PyTorch to implement the model.  We've provided a little bit of code to help with the data loading using PyTorch's data loaders (https://pytorch.org/docs/stable/data.html)

A model with the following architecture and hyperparameters should reach a validation perplexity below 226.
* embed the words with dimension 128, then flatten into a single embedding for $n-1$ words (with size $(n-1)*128$)
* run 2 hidden layers with 1024 hidden units, then project down to size 128 before the final layer (ie. 4 layers total). 
* use weight tying for the embedding and final linear layer (this made a very large difference in our experiments); you can do this by creating the output layer with `nn.Linear`, then using `F.embedding` with the linear layer's `.weight` to embed the input
* rectified linear activation (ReLU) and dropout 0.1 after first 2 hidden layers. **Note: You will likely find a performance drop if you add a nonlinear activation function after the dimension reduction layer.**
* train for 10 epochs with the Adam optimizer (should take around 15-20 minutes)
* do early stopping based on validation set perplexity (see Project 0)


We encourage you to try other architectures and hyperparameters, and you will likely find some that work better than the ones listed above.  A proper implementation with these should be enough to receive full credit on the assignment, though.

In [6]:
def ids(tokens):
    return [vocab.stoi[t] for t in tokens]

assert torch.cuda.is_available(), "no GPU found, in Colab go to 'Edit->Notebook settings' and choose a GPU hardware accelerator"

class NeuralNgramDataset(torch.utils.data.Dataset):
    def __init__(self, text_token_ids, n):
        self.text_token_ids = text_token_ids
        self.n = n

    def __len__(self):
        return len(self.text_token_ids)

    def __getitem__(self, i):
        if i < self.n-1:
            prev_token_ids = [vocab.stoi['<eos>']] * (self.n-i-1) + self.text_token_ids[:i]
        else:
            prev_token_ids = self.text_token_ids[i-self.n+1:i]

        assert len(prev_token_ids) == self.n-1

        x = torch.tensor(prev_token_ids)
        y = torch.tensor(self.text_token_ids[i])
        return x, y



class NeuralNGramNetwork(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.n = n
        self.embedding = nn.Embedding(len(vocab.itos), 128)
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * (n - 1), 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 128),
        )

    def forward(self, x):
        # x is a tensor of inputs with shape (batch, n-1)
        # this function returns a tensor of log probabilities with shape (batch, vocab_size)
        x_embed = self.embedding(x)  # (batch, n-1, 128)
        x_embed = x_embed.view(x_embed.size(0), -1)  # (batch, (n-1)*128)
        x_embed = self.fc_layers(x_embed)
        output = F.linear(x_embed, self.embedding.weight)  # (batch, vocab_size)
        return output



class NeuralNGramModel:
    # a class that wraps NeuralNGramNetwork to handle training and evaluation
    # it's ok if this doesn't work for unigram modeling
    def __init__(self, n):
        self.n = n
        self.network = NeuralNGramNetwork(n).cuda()


    def train(self):
        dataset = NeuralNgramDataset(ids(train_text), self.n)
        train_loader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)
        # iterating over train_loader with a for loop will return a 2-tuple of batched tensors
        # the first tensor will be previous token ids with size (batch, n-1),
        # and the second will be the current token id with size (batch, )
        # you will need to move these tensors to GPU, e.g. by using the Tensor.cuda() function.

        # this will take some time to run; use tqdm.tqdm_notebook to get a progress bar 
        # (see Project 1a for example)

        # YOUR CODE HERE
        optimizer = torch.optim.Adam(self.network.parameters())
        criterion = nn.CrossEntropyLoss()
        best_perplexity=float('inf')
        for epoch in range(10):
            for batch in train_loader:
                
                x,label=batch
                x,label=x.cuda(),label.cuda()
                optimizer.zero_grad()
                output=self.network(x)
                loss=criterion(output,label)
                loss.backward()
                optimizer.step()
            validation_perplexity = self.perplexity(validation_text)
            print('Epoch'+str(epoch)+' validation perplexity:', validation_perplexity)

            if validation_perplexity < best_perplexity:
                best_perplexity = validation_perplexity
                print("Best perplexity so far:",best_perplexity)
                torch.save(self.network.state_dict(), './tmp/nn_best.pt')

        self.network.load_state_dict(torch.load('./tmp/nn_best.pt'))
        return self.network
        
        
            

    def next_word_probabilities(self, text_prefix):
        # YOUR CODE HERE
        # don't forget self.network.eval()
        # you will need to convert text_prefix from strings to numbers with the `ids` function
        # if your `perplexity` function below is based on a NeuralNgramDataset DataLoader, you will need to use the same strategy for prefixes with less than n-1 tokens to pass the validity check
        #   the data loader appends extra "<eos>" (end of sentence) tokens to the start of the input so there are always enough to run the network
        if len(text_prefix) < self.n - 1:
            text_prefix = ['<eos>'] * (self.n-len(text_prefix)-1) + text_prefix
        text_prefix = text_prefix[-(self.n - 1):]
        prefix = torch.tensor(ids(text_prefix), dtype=torch.int64).cuda().view(1, -1)
        with torch.no_grad():
            self.network.eval()
            logits = self.network(prefix)
            self.network.train()
            probs = F.softmax(logits, dim=-1).view(-1)
        return probs.cpu().numpy()
        

    
    def perplexity(self, text):
        # you may want to use a DataLoader here with a NeuralNgramDataset
        # don't forget self.network.eval()

        # YOUR CODE HERE
         with torch.no_grad():
            self.network.eval()
            dataset = NeuralNgramDataset(ids(text), self.n)
            data_loader = torch.utils.data.DataLoader(dataset, batch_size=128)
            log_probabilities = []
            for batch in data_loader:
                x, label = batch
                x=x.cuda()
                label=label.cuda()
                output = self.network(x)
                output = F.softmax(output, dim=1)
                log_probabilities = [math.log(prob[idx], 2) for prob, idx in zip(output, label)]

            
            self.network.train()
            return 2 ** -np.mean(log_probabilities)



neural_trigram_model = NeuralNGramModel(5)
neural_trigram_model.train()
print('neural trigram validation perplexity:', neural_trigram_model.perplexity(validation_text))

save_truncated_distribution(neural_trigram_model, 'neural_trigram_predictions.npy', short=False)

Epoch0 validation perplexity: 26.898996577357654
Best perplexity so far: 26.898996577357654
Epoch1 validation perplexity: 25.46535567897791
Best perplexity so far: 25.46535567897791
Epoch2 validation perplexity: 27.259348999520174
Epoch3 validation perplexity: 28.260338907296067
Epoch4 validation perplexity: 26.83769076515407
Epoch5 validation perplexity: 23.009794667950327
Best perplexity so far: 23.009794667950327
Epoch6 validation perplexity: 25.10098799344845
Epoch7 validation perplexity: 24.29034093180757
Epoch8 validation perplexity: 25.362116242400827
Epoch9 validation perplexity: 25.000168619430713
neural trigram validation perplexity: 23.009794667950327


  0%|          | 0/5000 [00:00<?, ?it/s]

saved neural_trigram_predictions.npy


Fill in your neural trigram perplexity.

<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Neural trigram validation perplexity: ***23.009794667950327***

Free up RAM.

In [7]:
# Delete model we don't need. 
del neural_trigram_model

### LSTM Model

For this stage of the project, you will implement an LSTM language model.

For recurrent language modeling, the data batching strategy is a bit different from what is used in some other tasks.  Sentences are concatenated together so that one sentence starts right after the other, and an unfinished sentence will be continued in the next batch.  We'll use the `torchtext` library to manage this batching for you.  To properly deal with this input format, you should save the last state of the LSTM from a batch to feed in as the first state of the next batch.  When you save state across different batches, you should call `.detach()` on the state tensors before the next batch to tell PyTorch not to backpropagate gradients through the state into the batch you have already finished (which will cause a runtime error).

We expect your model to reach a validation perplexity below 130.  The following architecture and hyperparameters should be sufficient to get there.
* 3 LSTM layers with 512 units
* dropout of 0.5 after each LSTM layer
* instead of projecting directly from the last LSTM output to the vocabulary size for softmax, project down to a smaller size first (e.g. 512->128->vocab_size). **NOTE: You may find that adding nonlinearities between these layers can hurt performance, try without first.**
* use the same weights for the embedding layer and the pre-softmax layer; dimension 128
* train with Adam (using default learning rates) for at least 20 epochs


In [9]:

class LSTMNetwork(nn.Module):
    # a PyTorch Module that holds the neural network for your model

    def __init__(self):
        super().__init__()

        # YOUR CODE HERE
        self.embedding = nn.Embedding(len(vocab.itos), 128)   #[32, 64, 128]
        self.lstm=nn.LSTM(input_size=128, hidden_size=512, num_layers=3, dropout=0.7)
        self.fc=nn.Linear(512,128)
        self.pre_softmax=nn.Linear(128,len(vocab.itos))

    
    #batch shape: torch.Size([32, 64]),batch_size=64,bptt_len=32
    def forward(self, x, state):
        """Compute the output of the network.
        
        Note: In the Pytorch LSTM tutorial, the state variable is named "hidden":
        https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

        The torch.nn.LSTM documentation is quite helpful:
        https://pytorch.org/docs/stable/nn.html#lstm
    
        x - a tensor of int64 inputs with shape (seq_len, batch)
        state - a tuple of two tensors with shape (num_layers, batch, hidden_size)
                representing the hidden state and cell state of the of the LSTM.
        returns a tuple with two elements:
          - a tensor of log probabilities with shape (seq_len, batch, vocab_size)
          - a state tuple returned by applying the LSTM.
        """

        # Note that the nn.LSTM module expects inputs with the sequence 
        # dimension before the batch by default.
        # In this case the dimensions are already in the right order, 
        # but watch out for this since sometimes people put the batch first

        # YOUR CODE HERE
        x=self.embedding(x)
        x,state=self.lstm(x,state)
        x=self.fc(x)
        x=self.pre_softmax(x)
        x=F.softmax(x,2)
        return x, state



class LSTMModel:
    "A class that wraps LSTMNetwork to handle training and evaluation."

    def __init__(self):
        self.network = LSTMNetwork().cuda()

    def train(self):
        train_iterator = data.BPTTIterator(train_dataset, batch_size=64, bptt_len=32, device='cuda')
        # Iterate over train_iterator with a for loop to get batches
        # each batch object has a .text and .target attribute with
        # token id tensors for the input and output respectively.

        # The initial state passed into the LSTM should be set to zero.

        # YOUR CODE HERE
        optimizer = torch.optim.Adam(self.network.parameters())
        criterion=nn.CrossEntropyLoss()
        best_perplexity = float('inf')
        for epoch in range(20):
            state=None
            for batch in train_iterator:
                text,target=batch.text,batch.target
                text,target=text.cuda(),target.cuda()
                #print("text shape:",text.shape)
                #print("target shape:",target.shape)
                optimizer.zero_grad()
                output,(h,c) = self.network(text,state)
                
                target=target.view(-1)
                output=output.view(-1,output.shape[-1])
                loss = criterion(output,target)
                loss.backward()
                optimizer.step()
                h=h.detach()
                c=c.detach()
                state=(h,c)
                
                
            validation_perplexity = self.dataset_perplexity(validation_dataset)
            print('Epoch '+str(epoch)+' validation perplexity:', validation_perplexity)

            if validation_perplexity < best_perplexity:
                best_perplexity = validation_perplexity
                print("Best perplexity so far:",best_perplexity)
                torch.save(self.network.state_dict(), './tmp/lstm_best.pt')

        self.network.load_state_dict(torch.load('./tmp/lstm_best.pt'))
        return self.network
                
                
                


    def next_word_probabilities(self, text_prefix):
        "Return a list of probabilities for each word in the vocabulary."

        prefix_token_tensor = torch.tensor(ids(text_prefix), device='cuda' if torch.cuda.is_available() else 'cpu').view(-1, 1)
        
        # YOUR CODE HERE
        state = None
        with torch.no_grad(): 
            self.network.eval()
            output, state = self.network(prefix_token_tensor,state)
            prob = F.softmax(output[-1], 1).squeeze().tolist()
            self.network.train()
            return prob



    def dataset_perplexity(self, torchtext_dataset):
        "Return perplexity as a float."
        # Your code should be very similar to next_word_probabilities, but
        # run in a loop over batches. Use torch.no_grad() for extra speed.

        iterator = data.BPTTIterator(torchtext_dataset, batch_size=64, bptt_len=32, device='cuda')

        # YOUR CODE HERE
        state=None
        with torch.no_grad():
            self.network.eval()
            log_probabilities = []
            for batch in iterator:
                text, target = batch.text,batch.target
                text,target=text.cuda(),target.cuda()
                output,_= self.network(text,state)
                output = F.softmax(output, dim=1)
                target=target.view(-1)
                output=output.view(-1,output.shape[-1])
                log_probabilities = [math.log(prob[idx],2) for prob, idx in zip(output, target)]

            self.network.train()
            return 2 ** -np.mean(log_probabilities)




lstm_model = LSTMModel()
lstm_model.train()

print('lstm validation perplexity:', lstm_model.dataset_perplexity(validation_dataset))
save_truncated_distribution(lstm_model, 'lstm_predictions.npy', short=False)

Epoch 0 validation perplexity: 63.88211406763718
Best perplexity so far: 63.88211406763718
Epoch 1 validation perplexity: 63.84354316094525
Best perplexity so far: 63.84354316094525
Epoch 2 validation perplexity: 63.940717028817204
Epoch 3 validation perplexity: 63.95265389254983
Epoch 4 validation perplexity: 63.922663124820296
Epoch 5 validation perplexity: 63.92196811548475
Epoch 6 validation perplexity: 63.921176639254405
Epoch 7 validation perplexity: 63.92060441118135
Epoch 8 validation perplexity: 63.91997942058205
Epoch 9 validation perplexity: 63.921888212199384
Epoch 10 validation perplexity: 63.99521007876407
Epoch 11 validation perplexity: 63.94429598055493
Epoch 12 validation perplexity: 63.94805008721357
Epoch 13 validation perplexity: 63.95093707534752
Epoch 14 validation perplexity: 63.95858997055833
Epoch 15 validation perplexity: 63.962247395089406
Epoch 16 validation perplexity: 63.96525539810172
Epoch 17 validation perplexity: 63.96876650762371
Epoch 18 validation p

  0%|          | 0/5000 [00:00<?, ?it/s]

saved lstm_predictions.npy


<!-- Do not remove this comment, it is used by the autograder: RqYJKsoTS6 -->

Fill in your LSTM perplexity. 

LSTM validation perplexity: ***63.84354316094525***

### Submission

Upload a submission with the following files:
* hw1b.ipynb (rename to match this exactly)
* bigram_predictions.npy
* neural_trigram_predictions.npy
* (extension) lstm_predictions.npy (this should also include all improvements from your exploration) 
* report.pdf （can be markdown)

You can upload files individually or as part of a zip file, but if using a zip file be sure you are zipping the files directly and not a folder that contains them.

Be sure to check the output of the autograder after it runs.  It should confirm that no files are missing and that the output files have the correct format.  Note that the test set perplexities shown by the autograder are on a completely different scale from your validation set perplexities due to truncating the distribution and selecting different text.  Don't worry if the values seem much worse.